# Задание 1. Прямые методы решения линейных систем

In [1]:
from latexifier import latexify
from IPython.display import display, Markdown
import numpy as np
from typing import Callable, List, Tuple
import warnings
warnings.filterwarnings('ignore')

In [24]:
class Matrix(np.ndarray):
    """extends np.ndarray"""

    def __new__(cls, input_array):
        obj = np.asarray(input_array).view(cls)
        return obj

    def inverse_rows(self, i: int, j: int):
        """Swaps i-th and j-th rows of a matrix"""
        self[i], self[j] = self[j].copy(), self[i].copy()

    def inverse_cols(self, i: int, j: int):
        """Swaps i-th and j-th columns of a matrix"""
        self = self.transpose()
        self.inverse_rows(i, j)
        self = self.transpose()
    
    def tex(self) -> str:
        """Latexifies the matrix rounding up values up to 6 decimal values"""
        return latexify(self)

def append(M: np.ndarray, N: np.ndarray) -> Matrix:
    """Appends two numpy arrays with axis=1"""
    return Matrix(np.append(arr=np.asarray(M), values=N, axis=1))

def print_tex(*argv) -> None:
    """Displays a LaTex Markdown output"""
    res = ' '.join(['$$'] + [arg for arg in argv] + ['$$'])
    display(Markdown(res))

def get_latex_column(arg: str, n: int) -> str:
    """Prepares a string with LaTex syntax for a column of {arg} with indeces form 1 to {n}"""
    res = [r"""\begin{pmatrix}""", "\n"]
    for i in range(n):
        line = arg + "_{" + str(i+1) + "}" + (r"\\" if i != n-1 else "") + "\n"
        res.append(line)
    res.append(r"""\end{pmatrix}""")
    return ''.join(res)

def get_exact_column(col: Matrix, n: int) -> str:
    """Prepares a string with LaTex syntax for a column of {col} elements"""
    res = [r"""\begin{pmatrix}""", "\n"]
    for i in range(n):
        line = str(float(col[i][0])) + (r"\\" if i != n-1 else "") + "\n"
        res.append(line)
    res.append(r"""\end{pmatrix}""")
    return ''.join(res)

def print_system_output(A: Matrix, B: Matrix, X: Matrix, n: int) -> None:
    B_ = np.matmul(A, X)
    print_tex('X^* = ', get_latex_column("x^*", n), '=', X.tex())
    print_tex(r'A \times X^* = ', A.tex(), X.tex(), '=', B_.tex(), '= B^*')
    print_tex('B = ', get_exact_column(B, n), r'\stackrel{?}{=}', get_exact_column(B_, n), '= B^*')
    print_tex(r'B - B^* = ', get_exact_column(B - B_, n))


In [3]:
s = get_latex_column("x", 6)
print_tex(s, s)

$$ \begin{pmatrix}
x_{1}\\
x_{2}\\
x_{3}\\
x_{4}\\
x_{5}\\
x_{6}
\end{pmatrix} \begin{pmatrix}
x_{1}\\
x_{2}\\
x_{3}\\
x_{4}\\
x_{5}\\
x_{6}
\end{pmatrix} $$

In [7]:
n = int(input()) # Размерность матрицы A
A = Matrix(np.random.rand(n, n) * 10)
B = Matrix(np.random.rand(n, 1) * 10)
print_tex('A = ', A.tex(), ',~~', 'B = ', B.tex())
print_tex(A.tex(), get_latex_column('x', n), '=', B.tex())

$$ A =  \begin{pmatrix} 3.075671 & 7.985925 & 5.101466 & 4.084721 & 4.161815 \\ 1.604708 & 2.399418 & 4.038538 & 4.617231 & 8.351272 \\ 5.471106 & 6.244725 & 8.239714 & 4.990415 & 4.785699 \\ 9.225343 & 4.741391 & 9.378949 & 6.428078 & 5.937248 \\ 0.206587 & 9.518223 & 2.921500 & 6.723688 & 7.267771 \end{pmatrix} ,~~ B =  \begin{pmatrix} 7.532781 \\ 8.395154 \\ 7.402072 \\ 6.335163 \\ 5.678648 \end{pmatrix} $$

$$ \begin{pmatrix} 3.075671 & 7.985925 & 5.101466 & 4.084721 & 4.161815 \\ 1.604708 & 2.399418 & 4.038538 & 4.617231 & 8.351272 \\ 5.471106 & 6.244725 & 8.239714 & 4.990415 & 4.785699 \\ 9.225343 & 4.741391 & 9.378949 & 6.428078 & 5.937248 \\ 0.206587 & 9.518223 & 2.921500 & 6.723688 & 7.267771 \end{pmatrix} \begin{pmatrix}
x_{1}\\
x_{2}\\
x_{3}\\
x_{4}\\
x_{5}
\end{pmatrix} = \begin{pmatrix} 7.532781 \\ 8.395154 \\ 7.402072 \\ 6.335163 \\ 5.678648 \end{pmatrix} $$

### Решение методом Гаусса единственного деления

In [25]:
def SingleGauss(A: Matrix, B: Matrix, n: int) -> Matrix:
    A_ = A.copy()
    B_ = B.copy()
    X = [[0] for _ in range(n)]
    for k in range(n):
        for j in range(k+1, n):
            d = A_[j][k] / A_[k][k]
            for i in range(k, n):
                A_[j][i] = A_[j][i] - d * A_[k][i]
            B_[j] -= d * B_[k][0]
    for k in range(n-1, -1, -1):
        d = 0
        for j in range(k, n):
            s = A_[k][j] * X[j][0]
            d += s
        X[k][0] = (B_[k][0] - d) / A_[k][k]
    return Matrix(np.array(X))


print_system_output(A, B, SingleGauss(A, B, n), n)


$$ X^* =  \begin{pmatrix}
x^*_{1}\\
x^*_{2}\\
x^*_{3}\\
x^*_{4}\\
x^*_{5}
\end{pmatrix} = \begin{pmatrix} 0.292273 \\ 0.820207 \\ 0.567855 \\ -2.600989 \\ 1.876862 \end{pmatrix} $$

$$ A \times X^* =  \begin{pmatrix} 3.075671 & 7.985925 & 5.101466 & 4.084721 & 4.161815 \\ 1.604708 & 2.399418 & 4.038538 & 4.617231 & 8.351272 \\ 5.471106 & 6.244725 & 8.239714 & 4.990415 & 4.785699 \\ 9.225343 & 4.741391 & 9.378949 & 6.428078 & 5.937248 \\ 0.206587 & 9.518223 & 2.921500 & 6.723688 & 7.267771 \end{pmatrix} \begin{pmatrix} 0.292273 \\ 0.820207 \\ 0.567855 \\ -2.600989 \\ 1.876862 \end{pmatrix} = \begin{pmatrix} 7.532781 \\ 8.395154 \\ 7.402072 \\ 6.335163 \\ 5.678648 \end{pmatrix} = B^* $$

$$ B =  \begin{pmatrix}
7.532781400947558\\
8.395153646790689\\
7.402072215987003\\
6.335162991184713\\
5.678647791700553
\end{pmatrix} \stackrel{?}{=} \begin{pmatrix}
7.532781400947559\\
8.39515364679069\\
7.4020722159870065\\
6.335162991184713\\
5.678647791700525
\end{pmatrix} = B^* $$

$$ B - B^* =  \begin{pmatrix}
-8.881784197001252e-16\\
-1.7763568394002505e-15\\
-3.552713678800501e-15\\
0.0\\
2.7533531010703882e-14
\end{pmatrix} $$

### Решение методом Гаусса с выбором главного элемента

In [26]:
def ModifiedGauss(A: Matrix, B: Matrix, n: int) -> Matrix:
    AB = append(A, B)
    X = [[0] for _ in range(n)]
    for k in range(n):
        p = k # главный элемент
        for m in range(k+1, n):
            if abs(AB[p][k]) < abs(AB[m][k]):
                p = m # индекс строки, содержащей максимальный элемент в столбце
        AB.inverse_rows(k, p) # перестановка строк
        for m in range(k+1, n):
            c = AB[m][k] / AB[k][k]
            for i in range(n+1):
                AB[m][i] -= c * AB[k][i]
        X[n-1][0] = AB[n-1][n] / AB[n-1][n-1]
        for k in range(n-1, -1, -1):
            s = 0
            for i in range(k+1, n):
                s += AB[k][i] * X[i][0]
            X[k][0] = (AB[k][n] - s) / AB[k][k]
    return Matrix(np.array(X))

print_system_output(A, B, ModifiedGauss(A, B, n), n)

$$ X^* =  \begin{pmatrix}
x^*_{1}\\
x^*_{2}\\
x^*_{3}\\
x^*_{4}\\
x^*_{5}
\end{pmatrix} = \begin{pmatrix} 0.292273 \\ 0.820207 \\ 0.567855 \\ -2.600989 \\ 1.876862 \end{pmatrix} $$

$$ A \times X^* =  \begin{pmatrix} 3.075671 & 7.985925 & 5.101466 & 4.084721 & 4.161815 \\ 1.604708 & 2.399418 & 4.038538 & 4.617231 & 8.351272 \\ 5.471106 & 6.244725 & 8.239714 & 4.990415 & 4.785699 \\ 9.225343 & 4.741391 & 9.378949 & 6.428078 & 5.937248 \\ 0.206587 & 9.518223 & 2.921500 & 6.723688 & 7.267771 \end{pmatrix} \begin{pmatrix} 0.292273 \\ 0.820207 \\ 0.567855 \\ -2.600989 \\ 1.876862 \end{pmatrix} = \begin{pmatrix} 7.532781 \\ 8.395154 \\ 7.402072 \\ 6.335163 \\ 5.678648 \end{pmatrix} = B^* $$

$$ B =  \begin{pmatrix}
7.532781400947558\\
8.395153646790689\\
7.402072215987003\\
6.335162991184713\\
5.678647791700553
\end{pmatrix} \stackrel{?}{=} \begin{pmatrix}
7.532781400947558\\
8.395153646790689\\
7.402072215987001\\
6.335162991184713\\
5.678647791700554
\end{pmatrix} = B^* $$

$$ B - B^* =  \begin{pmatrix}
0.0\\
0.0\\
1.7763568394002505e-15\\
0.0\\
-8.881784197001252e-16
\end{pmatrix} $$

### Решение методом LU-разложения

In [20]:
def GetLU(A: Matrix, n: int) -> Tuple[Matrix, Matrix]:
    L = Matrix(np.zeros(shape=(n, n)))
    U = A.copy()
    for i in range(n):
        for j in range(i, n):
            L[j][i] = U[j][i] / U[i][i]
    for k in range(1, n):
        for i in range(k-1, n):
            for j in range(i, n):
                L[j][i] = U[j][i] / U[i][i]
        for i in range(k, n):
            for j in range(k-1, n):
                U[i][j] = U[i][j] - L[i][k-1] * U[k-1][j]

    return L, U

L, U = GetLU(A, n)
print_tex('L = ', L.tex())
print_tex('U = ', U.tex())
print_tex(r'L \times U = ', (np.matmul(L, U)).tex(), r'\stackrel{?}{=}', A.tex(), '= A')

$$ L =  \begin{pmatrix} 1 & 0 & 0 & 0 & 0 \\ 0.521742 & 1 & 0 & 0 & 0 \\ 1.778833 & 4.504871 & 1 & 0 & 0 \\ 2.999457 & 10.871606 & 2.968559 & 1 & 0 \\ 0.067168 & -5.082584 & -1.360824 & 0.104592 & 1 \end{pmatrix} $$

$$ U =  \begin{pmatrix} 3.075671 & 7.985925 & 5.101466 & 4.084721 & 4.161815 \\ 0 & -1.767176 & 1.376888 & 2.486060 & 6.179878 \\ 0 & 0 & -7.037643 & -13.474999 & -30.457025 \\ 0 & 0 & 0 & 7.150000 & 16.682339 \\ 0 & 0 & 0 & 0 & -4.793507 \end{pmatrix} $$

$$ L \times U =  \begin{pmatrix} 3.075671 & 7.985925 & 5.101466 & 4.084721 & 4.161815 \\ 1.604708 & 2.399418 & 4.038538 & 4.617231 & 8.351272 \\ 5.471106 & 6.244725 & 8.239714 & 4.990415 & 4.785699 \\ 9.225343 & 4.741391 & 9.378949 & 6.428078 & 5.937248 \\ 0.206587 & 9.518223 & 2.921500 & 6.723688 & 7.267771 \end{pmatrix} \stackrel{?}{=} \begin{pmatrix} 3.075671 & 7.985925 & 5.101466 & 4.084721 & 4.161815 \\ 1.604708 & 2.399418 & 4.038538 & 4.617231 & 8.351272 \\ 5.471106 & 6.244725 & 8.239714 & 4.990415 & 4.785699 \\ 9.225343 & 4.741391 & 9.378949 & 6.428078 & 5.937248 \\ 0.206587 & 9.518223 & 2.921500 & 6.723688 & 7.267771 \end{pmatrix} = A $$

In [27]:
Y_ = SingleGauss(L, B, n)
X_ = SingleGauss(U, Y_, n)
print_system_output(A, B, X_, n)

$$ X^* =  \begin{pmatrix}
x^*_{1}\\
x^*_{2}\\
x^*_{3}\\
x^*_{4}\\
x^*_{5}
\end{pmatrix} = \begin{pmatrix} 0.292273 \\ 0.820207 \\ 0.567855 \\ -2.600989 \\ 1.876862 \end{pmatrix} $$

$$ A \times X^* =  \begin{pmatrix} 3.075671 & 7.985925 & 5.101466 & 4.084721 & 4.161815 \\ 1.604708 & 2.399418 & 4.038538 & 4.617231 & 8.351272 \\ 5.471106 & 6.244725 & 8.239714 & 4.990415 & 4.785699 \\ 9.225343 & 4.741391 & 9.378949 & 6.428078 & 5.937248 \\ 0.206587 & 9.518223 & 2.921500 & 6.723688 & 7.267771 \end{pmatrix} \begin{pmatrix} 0.292273 \\ 0.820207 \\ 0.567855 \\ -2.600989 \\ 1.876862 \end{pmatrix} = \begin{pmatrix} 7.532781 \\ 8.395154 \\ 7.402072 \\ 6.335163 \\ 5.678648 \end{pmatrix} = B^* $$

$$ B =  \begin{pmatrix}
7.532781400947558\\
8.395153646790689\\
7.402072215987003\\
6.335162991184713\\
5.678647791700553
\end{pmatrix} \stackrel{?}{=} \begin{pmatrix}
7.532781400947558\\
8.395153646790689\\
7.402072215987002\\
6.335162991184699\\
5.678647791700532
\end{pmatrix} = B^* $$

$$ B - B^* =  \begin{pmatrix}
0.0\\
0.0\\
8.881784197001252e-16\\
1.4210854715202004e-14\\
2.1316282072803006e-14
\end{pmatrix} $$